In [1]:
import gradio

/home/bachngo/anaconda3/envs/gemm_assistant/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import requests

# Define the paper search endpoint URL
url = 'https://api.semanticscholar.org/graph/v1/paper/search'

# Define the required query parameter and its value (in this case, the keyword we want to search for)
query_params = {
    'query': 'semantic scholar platform',
    'limit': 1
}

# Make the GET request with the URL and query parameters
search_response = requests.get(url, params=query_params)

In [44]:
# Define a separate function to make a request to the paper details endpoint using a paper_id. This function will be used later on (after we call the paper search endpoint).
def get_paper_data(paper_ids):
  url = 'https://api.semanticscholar.org/graph/v1/paper/batch'

  # Define which details about the paper you would like to receive in the response
  paper_data_query_params = {'fields': 'references'}

  # Send the API request and store the response in a variable
  response = requests.post(url, params=paper_data_query_params, json={"ids": paper_ids})
  if response.status_code == 200:
    return response.json()
  else:
    return None

# Retrieve the paper id corresponding to the 1st result in the list
paper_id = ["arxiv:2204.03458", "arxiv:2405.04133"]

# Retrieve the paper details corresponding to this paper id using the function we defined earlier.
paper_details = get_paper_data(paper_id)

In [51]:
import networkx as nx

class PaperNode:
    title: str
    arxiv_id: str
    
    def __init__(self, title, arxiv_id):
        self.title = title
        self.arxiv_id = arxiv_id

    def __str__(self) -> str:
        return f"Title: {self.title},\n Arxiv ID: {self.arxiv_id}"

class PaperEdge:
    category: str
    explanation: str
    verbose = True

    def __init__(self, category, explanation):
        self.category = category
        self.explanation = explanation

    def __str__(self) -> str:
        if self.verbose:
            return f"Category: {self.category},\n Explanation: {self.explanation}"
        else:
            return f"Category: {self.category}"

def create_ego_graph(retriever_response, service: str = "ss", graph: nx.DiGraph = None) -> nx.DiGraph:

    if service == "ss":
        paper_ids = ["arxiv:"+ n.metadata["paper_id"] for n in retriever_response]
        url = 'https://api.semanticscholar.org/graph/v1/paper/batch'

        # Define which details about the paper you would like to receive in the response
        paper_data_query_params = {'fields': 'references'}
        G = nx.DiGraph()

        # Send the API request and store the response in a variable
        response = requests.post(url, params=paper_data_query_params, json={"ids": paper_ids})
        source_nodes = []
        if response.status_code == 200:
            data = response.json()
            for idx, item in enumerate(data):
                source_node = PaperNode(
                    title=retriever_response[idx].metadata["title"],
                    arxiv_id=retriever_response[idx].metadata["paper_id"]
                )
                source_nodes.append(source_node)
                for reference in item["references"]:
                    target_node = PaperNode(
                        title=reference["title"],
                        arxiv_id=reference["paperId"]
                    )
                    
                    G.add_edge(source_node.title, target_node.title)
            
            nodes_to_remove = [node for node in G if G.degree(node) < 2]

            # Remove the nodes from the ego graph
            G.remove_nodes_from(nodes_to_remove)
            # Assign colors: highlighted nodes in red, others in blue
            highlight_color = "orange"
            for node in G.nodes():
                if node in source_nodes:
                    G.nodes[node]['color'] = highlight_color
                    
            return G
        else:
            raise ValueError(f"Request failed with status code {response.status_code}.")

[{'paperId': '4247f45a5730e3bda5836e2bc7941e30f5b91cb7', 'title': 'Board'},
 {'paperId': 'af9f365ed86614c800f082bd8eb14be76072ad16',
  'title': 'Classifier-Free Diffusion Guidance'},
 {'paperId': '4e65a4f248344c0a17e20d712d38d91085f608a2',
  'title': 'Transframer: Arbitrary Frame Prediction with Generative Models'},
 {'paperId': '1641774b55a471a23eb31b722ee05c2e032fec7a',
  'title': 'Diffusion Probabilistic Modeling for Video Generation'},
 {'paperId': '7002ae048e4b8c9133a55428441e8066070995cb',
  'title': 'GLIDE: Towards Photorealistic Image Generation and Editing with Text-Guided Diffusion Models'},
 {'paperId': '9eeb5f7c36654dff1dc93adb7150e7bab52cd3e2',
  'title': 'Deblurring via Stochastic Refinement'},
 {'paperId': '97bee918b08c244eb2e54d41e8ea6da00a3e5dbf',
  'title': 'NÜWA: Visual Synthesis Pre-training for Neural visUal World creAtion'},
 {'paperId': '37c9c4e7648f639c0b36f150fc6c6c90b3682f4a',
  'title': 'Palette: Image-to-Image Diffusion Models'},
 {'paperId': '5e51ebd5507cf6

In [19]:
import requests
import feedparser

def get_latest_arxiv_papers(categories=['cs.AI', 'cs.CV', 'cs.IR', 'cs.LG', 'cs.CL'], max_results=50):
    base_url = 'http://export.arxiv.org/api/query?'
    all_categories=[f'cat:{category}' for category in categories]
    search_query = '+OR+'.join(all_categories)
    query = f'search_query={search_query}&start=0&max_results={max_results}&sortBy=submittedDate&sortOrder=descending'
    response = requests.get(base_url + query)
    feed = feedparser.parse(response.content)
    return feed.entries

response = get_latest_arxiv_papers()

cat:cs.AI+OR+cat:cs.CV+OR+cat:cs.IR+OR+cat:cs.LG+OR+cat:cs.CL


In [27]:
template = """
I am interested in LLM, Generative AI and trendy stuffs in AI, picks out the most notable papers published today:

==============
{paper_list}

"""

paper_list = []

for r in response:
    paper_list.append(f"""
Title: {r['title']}
Link: {r['link']}
Summary: {r['summary']}
                  
    """)

prompt = prompt = template.format(paper_list="\n==============\n".join(paper_list))

In [31]:
from llama_index.llms.openai import OpenAI
import os
import dotenv
dotenv.load_dotenv(override=True)

llm = OpenAI(model="gpt-4o", api_key=os.environ["OPENAI_API_KEY"])
res = llm.complete(prompt)

In [33]:
print(res)

Here are some of the most notable papers published today in the fields of Large Language Models (LLMs), Generative AI, and other trendy AI topics:

1. **MambaOut: Do We Really Need Mamba for Vision?**
   - **Link:** [arxiv.org/abs/2405.07992v1](http://arxiv.org/abs/2405.07992v1)
   - **Summary:** This paper explores the necessity of the Mamba architecture for vision tasks, concluding that Mamba is not essential for image classification but shows potential for long-sequence visual tasks like detection and segmentation.

2. **SPIN: Simultaneous Perception, Interaction and Navigation**
   - **Link:** [arxiv.org/abs/2405.07991v1](http://arxiv.org/abs/2405.07991v1)
   - **Summary:** Introduces a reactive mobile manipulation framework that integrates perception, interaction, and navigation, allowing a mobile manipulator to navigate and interact with its environment using an active visual system.

3. **Plot2Code: A Comprehensive Benchmark for Evaluating Multi-modal Large Language Models in Co